Problem Description

How to establish the best London subway startions with the highest density of amenities around the given stations in the London area
The goal is to understand where is has the highest number of eateries an other amenities required to satisfy all the necessities of life in a densely 
populated area such as inner London both for residents and tourists alike.

Who could be interested to learn about this?

This is  targeting both residnets and tourists who are looking to find apartments or houses in broader London area that
have the highest level of amenities around train stations

Why should this be relevant for the target group?

Finding a variety of amenities (eateries, convenience stores working 24/7, shops of all sorts) can be a  frustartining, so the key is 
a quick access around the trasnport grid that is vital both  for the busy  workforce and studnets, but also tourists who also are not familiar with the neighborhood in inner London
they are in at the moment, and they very often rely on the subway or trains because not feeling to be in charge of the unknown space.
So, this little project should should help interested parties who are loking for places aroudn the tarnsportataion where there are stilll opprtunities for Business development,
and on the other side for the people who are looking for the aformentioned quick access to teh city and all it has to offer.

This solution can be extended to any neighbourhood in any other big city.


Letus first import all the relevant libraries

In [85]:
import pandas as pd
import numpy as np
import folium

import requests
import lxml

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Let us access the required dataset

In [86]:
London_stations = pd.read_csv('https://www.doogal.co.uk/LondonStationsCSV.ashx')

In [87]:
London_stations.head(n=20)

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508758,-0.263430,2,W3 6BD
3,Acton Main Line,520296,181196,51.516887,-0.267690,3,W3 0BP
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN
5,Addington Village,537082,163744,51.356239,-0.032665,"3,4,5,6",CR0 5AR
6,Addiscombe,534190,166290,51.379808,-0.073213,"3,4,5,6",CR0 7AA
7,Albany Park,547903,172902,51.435816,0.126445,5,DA5 3HP
8,Aldgate,533629,181246,51.514342,-0.075627,1,EC3N 1AH
9,Aldgate East,533809,181333,51.515082,-0.073001,1,E1 7PT


In [53]:
London_stations.drop(columns=['OS X', 'OS Y', 'Postcode'], axis=1, inplace=True)

Our Focus London Station Table and filter for zones 1 and 2 ( so we avoid troubles with Foursquare)

In [88]:
London_stations = London_stations[London_stations['Zone'].isin(['1', '2'])]

In [56]:
London_stations.reset_index(inplace=True)

In [57]:
London_stations.drop(columns=['index'], inplace=True)

So let us plot all the statsions in Zone 1 and 2 and see how theu are positioned in inner London

In [67]:
# create map of London using latitude and longitude values
London_lat = 51.5074
London_long = -0.1278
map_London = folium.Map(location=[London_lat, London_long], zoom_start=11)

# add markers to map
for lat, lng, label in zip(London_stations['Latitude'], London_stations['Longitude'], London_stations['Station']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  


map_London

Let us engage FoursquareAPI here

In [59]:
CLIENT_ID = 'SAYYCP1RPZPYMGJYQO2TOPF4DIDHM0MKL3WBBYGRCS3X01IF' # your Foursquare ID
CLIENT_SECRET = 'C1B55PLBYAWFMSBVCVGWVXQHHSMDWDHDWXQOX1CQ5KGQ5HHM' # your Foursquare Secret
VERSION = '20190120' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SAYYCP1RPZPYMGJYQO2TOPF4DIDHM0MKL3WBBYGRCS3X01IF
CLIENT_SECRET:C1B55PLBYAWFMSBVCVGWVXQHHSMDWDHDWXQOX1CQ5KGQ5HHM


Now  we  should cretae  getnerabyvenues function to obtain venues from FourSquare App

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Lo lets us make a query and have all the statstions listed

In [62]:
# type your answer here
LIMIT=100
London_venues = getNearbyVenues(names=London_stations['Station'],
                                   latitudes=London_stations['Latitude'],
                                   longitudes=London_stations['Longitude']
                                  )

Acton Central
Aldgate
Aldgate East
All Saints
Angel
Arsenal
Baker Street
Bank
Barbican
Barons Court
Battersea Park
Bayswater
Belsize Park
Bermondsey
Bethnal Green
Bethnal Green Rail
Blackfriars
Blackwall
Bond Street
Borough
Bow Church
Bow Road
Brixton
Brockley
Brondesbury
Brondesbury Park
Caledonian Road
Caledonian Road and Barnsbury
Cambridge Heath
Camden Road
Camden Town
Canada Water
Canary Wharf
Cannon Street
Canonbury
Chalk Farm
Chancery Lane
Charing Cross
City Thameslink
Clapham Common
Clapham High Street
Clapham Junction
Clapham North
Clapton
Covent Garden
Crossharbour and London Arena
Cutty Sark for Maritime Greenwich
Dalston Junction
Dalston Kingsland
Denmark Hill
Deptford
Devons Road
Drayton Park
East Acton
East Dulwich
Edgware Road (Bakerloo)
Edgware Road (Circle/District/Hammersmith and City)
Embankment
Essex Road
Euston
Euston Square
Farringdon
Fenchurch Street
Finchley Road
Finchley Road and Frognal
Finsbury Park
Fulham Broadway
Gloucester Road
Goldhawk Road
Goodge Street


In [63]:
London_venues.head()

,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton Central,51.508758,-0.26343,The Station House,51.508877,-0.263076,Pub
1,Acton Central,51.508758,-0.26343,Acton Park,51.508595,-0.261573,Park
2,Acton Central,51.508758,-0.26343,The Rocket,51.508772,-0.263787,Pub
3,Acton Central,51.508758,-0.26343,Laveli Bakery,51.508859,-0.263366,Bakery
4,Acton Central,51.508758,-0.26343,Acton Central London Overground Station,51.508590,-0.262928,Train Station


Now we are seeing a sample for Action Central and the venue categories a round the given station, let us see how many venue categories there are.

In [64]:
London_venues.groupby('Station').count()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,,,,,,
Acton Central,6,6,6,6,6,6
Aldgate,29,29,29,29,29,29
Aldgate East,25,25,25,25,25,25
All Saints,6,6,6,6,6,6
Angel,32,32,32,32,32,32
Arsenal,5,5,5,5,5,5
Baker Street,32,32,32,32,32,32
Bank,19,19,19,19,19,19
Barbican,6,6,6,6,6,6


In [17]:
print('There are {} uniques categories.'.format(len(London_venues['Venue Category'].unique())))

There are 284 uniques categories.


We will now use One hot encoding - to perform “binarization” of the category and include it as a feature tobetetr understand and train our model.

In [65]:
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")
#london_onehot.drop(labels=['Station'], axis=1, inplace = True)
station = London_venues['Station']
London_onehot.insert(0, 'Station', station)

In [66]:
London_onehot.head()

,Station,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio
0,Acton Central,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acton Central,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acton Central,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acton Central,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton Central,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we need to group venues by the Station area they are in 

In [68]:
London_grouped = London_onehot.groupby('Station').mean().reset_index()
London_grouped

,Station,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio
0,Acton Central,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
1,Aldgate,0.0,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
2,Aldgate East,0.0,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.0000,0.040000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
3,All Saints,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
4,Angel,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.031250,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
5,Arsenal,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
6,Baker Street,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.031250,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
7,Bank,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.052632,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
8,Barbican,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00
9,Barons Court,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00


In order to not get lost in the plethora odf data we will focus on top 10 most common locations around every station.

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
London_venues_sorted = pd.DataFrame(columns=columns)
London_venues_sorted['Station'] = London_grouped['Station']

for ind in np.arange(london_grouped.shape[0]):
    London_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

London_venues_sorted

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Central,Pub,Park,Bakery,Train Station,Grocery Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
1,Aldgate,Pub,Sandwich Place,Hotel,Italian Restaurant,Coffee Shop,Sushi Restaurant,Mediterranean Restaurant,Gym / Fitness Center,Thai Restaurant,Cocktail Bar
2,Aldgate East,Pub,Coffee Shop,Italian Restaurant,Salad Place,Thai Restaurant,Sushi Restaurant,Sandwich Place,Chinese Restaurant,Restaurant,Grocery Store
3,All Saints,Chinese Restaurant,Pool,Bakery,Pizza Place,Café,Restaurant,Fish Market,Event Space,Falafel Restaurant,Farmers Market
4,Angel,Coffee Shop,Burrito Place,Supermarket,Nightclub,Garden,Chinese Restaurant,Sushi Restaurant,Clothing Store,Tea Room,Gym / Fitness Center
5,Arsenal,Park,Sports Bar,Fish & Chips Shop,Metro Station,Yoga Studio,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market
6,Baker Street,Coffee Shop,Museum,Pizza Place,Sushi Restaurant,Gastropub,Food & Drink Shop,English Restaurant,Outdoor Sculpture,Gift Shop,Chinese Restaurant
7,Bank,French Restaurant,Coffee Shop,Italian Restaurant,Candy Store,Gym / Fitness Center,Steakhouse,Lounge,Restaurant,Historic Site,History Museum
8,Barbican,Pub,Hotel,Coffee Shop,Museum,Gym / Fitness Center,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
9,Barons Court,Platform,Café,Gym,Dry Cleaner,Convenience Store,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


Now it is time to chose our ML model and we will go for k means clustering that will help us to better understand areas. We will go for 5 clusters (0-4)

In [73]:
# set number of clusters
kclusters = 5

London_grouped_clustering = London_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 2, 2, 0, 2, 2, 4, 3], dtype=int32)

In [75]:
London_merged = London_stations

# add clustering labels
London_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
London_merged = London_merged.join(London_venues_sorted.set_index('Station'), on='Station')

London_merged.head() # check the last columns!

,Station,Latitude,Longitude,Zone,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Central,51.508758,-0.263430,2,4,Pub,Park,Bakery,Train Station,Grocery Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
1,Aldgate,51.514342,-0.075627,1,4,Pub,Sandwich Place,Hotel,Italian Restaurant,Coffee Shop,Sushi Restaurant,Mediterranean Restaurant,Gym / Fitness Center,Thai Restaurant,Cocktail Bar
2,Aldgate East,51.515082,-0.073001,1,4,Pub,Coffee Shop,Italian Restaurant,Salad Place,Thai Restaurant,Sushi Restaurant,Sandwich Place,Chinese Restaurant,Restaurant,Grocery Store
3,All Saints,51.510477,-0.012625,2,2,Chinese Restaurant,Pool,Bakery,Pizza Place,Café,Restaurant,Fish Market,Event Space,Falafel Restaurant,Farmers Market
4,Angel,51.532968,-0.105581,1,2,Coffee Shop,Burrito Place,Supermarket,Nightclub,Garden,Chinese Restaurant,Sushi Restaurant,Clothing Store,Tea Room,Gym / Fitness Center


And here is the map of our Clusters

In [76]:
# create map
map_clusters = folium.Map(location=[London_lat, London_long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Latitude'], London_merged['Longitude'], London_merged['Station'],London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

is showing us that the most frequent venues are parks and there might be a place  for Business development (sport facilities could be an option)as this is our smallest Cluster and it shows that thee areas are not high on tourists agenda.

In [78]:
London_merged.loc[London_merged['Cluster Labels'] == 0, London_merged.columns[[0] + list(range(5, London_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Arsenal,Park,Sports Bar,Fish & Chips Shop,Metro Station,Yoga Studio,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market
15,Bethnal Green Rail,Park,Yoga Studio,Flea Market,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop
89,Island Gardens,Park,Café,Trail,Light Rail Station,Athletics & Sports,Food Stand,Food Court,Food Truck,Food & Drink Shop,Flower Shop
132,Pimlico,Park,Sandwich Place,Dry Cleaner,Fish Market,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


# Cluster 2

Cluseter 1 is listing hotels as most frequent ones, as this is the most frequented area by Tourists, so the market might be saturated for further development in the hopsitality filed . 

In [79]:
London_merged.loc[London_merged['Cluster Labels'] == 1, London_merged.columns[[0] + list(range(5, London_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Blackfriars,Italian Restaurant,Pub,Tapas Restaurant,Coffee Shop,Hotel,Vietnamese Restaurant,Bar,Japanese Restaurant,Dessert Shop,Dance Studio
17,Blackwall,Hotel,Light Rail Station,Grocery Store,Coffee Shop,Italian Restaurant,Pub,Food Stand,Food Court,Food & Drink Shop,Flower Shop
20,Bow Church,Pub,Hotel,Park,Bus Stop,Burger Joint,Locksmith,Convenience Store,Yoga Studio,Fish Market,Farmers Market
37,Charing Cross,Hotel,Wine Bar,Pub,Currywurst Joint,Liquor Store,Gay Bar,North Indian Restaurant,Garden,Sandwich Place,Bookstore
57,Embankment,Hotel,Pub,Garden,Boat or Ferry,Coffee Shop,Piano Bar,Mexican Restaurant,Sandwich Place,Sporting Goods Shop,Bar
62,Fenchurch Street,Hotel,Gym / Fitness Center,Café,Cocktail Bar,Restaurant,Steakhouse,Fast Food Restaurant,Historic Site,Sandwich Place,Modern European Restaurant
67,Gloucester Road,Hotel,Restaurant,Italian Restaurant,Bar,Mediterranean Restaurant,Pub,Coffee Shop,Residential Building (Apartment / Condo),Indian Restaurant,Asian Restaurant
72,Green Park,Hotel,Sandwich Place,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Modern European Restaurant,French Restaurant,Molecular Gastronomy Restaurant,Coffee Shop,Boutique
83,Holborn,Hotel,Restaurant,Coffee Shop,Pub,Gym / Fitness Center,Donut Shop,History Museum,Portuguese Restaurant,English Restaurant,Hotel Bar
86,Homerton,Hotel,Convenience Store,Bed & Breakfast,Yoga Studio,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market


# Cluster 3

This is  our  cluster with highest venue density-its an eatery hotspot , it is a  great spot for Tourists but not a winner in terms of Business development.

In [80]:
London_merged.loc[London_merged['Cluster Labels'] == 2, London_merged.columns[[0] + list(range(5, London_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,All Saints,Chinese Restaurant,Pool,Bakery,Pizza Place,Café,Restaurant,Fish Market,Event Space,Falafel Restaurant,Farmers Market
4,Angel,Coffee Shop,Burrito Place,Supermarket,Nightclub,Garden,Chinese Restaurant,Sushi Restaurant,Clothing Store,Tea Room,Gym / Fitness Center
6,Baker Street,Coffee Shop,Museum,Pizza Place,Sushi Restaurant,Gastropub,Food & Drink Shop,English Restaurant,Outdoor Sculpture,Gift Shop,Chinese Restaurant
7,Bank,French Restaurant,Coffee Shop,Italian Restaurant,Candy Store,Gym / Fitness Center,Steakhouse,Lounge,Restaurant,Historic Site,History Museum
12,Belsize Park,Pizza Place,Coffee Shop,Bakery,Hotel,Movie Theater,Bookstore,Metro Station,Convenience Store,Food & Drink Shop,Pharmacy
14,Bethnal Green,Coffee Shop,Park,Restaurant,Cocktail Bar,Yoga Studio,Beer Bar,Pub,Convenience Store,Church,Fast Food Restaurant
18,Bond Street,Coffee Shop,Café,Hotel,Italian Restaurant,Turkish Restaurant,Juice Bar,Cosmetics Shop,Sandwich Place,Shoe Store,Burger Joint
22,Brixton,Market,Pizza Place,Coffee Shop,Fried Chicken Joint,Beer Bar,Nightclub,Food Court,Mexican Restaurant,Tapas Restaurant,Cupcake Shop
23,Brockley,Coffee Shop,Convenience Store,Gastropub,Beer Store,Italian Restaurant,Train Station,Deli / Bodega,Pub,Restaurant,Pakistani Restaurant
25,Brondesbury Park,Snack Place,Train Station,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio


# Cluster 4

This Cluster is on a low point for Tourists and  there might be some opprtunities within Busines Development but not as high as Cluster 1 as this area is not really accessible and its rather suburbian, the devlopment should be focused on accomodating needs of  the area residents (i.e. no fast food restaurants in top 5 venues)

In [82]:
London_merged.loc[London_merged['Cluster Labels'] == 3, London_merged.columns[[0] + list(range(5, London_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Barons Court,Platform,Café,Gym,Dry Cleaner,Convenience Store,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
34,Canonbury,Platform,Park,Pub,Train Station,Coffee Shop,Fish Market,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
122,New Cross,Platform,Pub,Bus Stop,Cocktail Bar,Train Station,Turkish Restaurant,Yoga Studio,Fish Market,Falafel Restaurant,Farmers Market
124,Nunhead,Café,Garden Center,Platform,Yoga Studio,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
189,White City,Café,Park,Platform,Pizza Place,Metro Station,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


# Cluster 5 

This is an area with Pubs being nr one venue category and there are fast food restaurants so the venyues are caterinng obviously to Tourists and highly commuting workforce. However high density is not giving any room for 

In [83]:
London_merged.loc[London_merged['Cluster Labels'] == 4, London_merged.columns[[0] + list(range(5, London_merged.shape[1]))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Central,Pub,Park,Bakery,Train Station,Grocery Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
1,Aldgate,Pub,Sandwich Place,Hotel,Italian Restaurant,Coffee Shop,Sushi Restaurant,Mediterranean Restaurant,Gym / Fitness Center,Thai Restaurant,Cocktail Bar
2,Aldgate East,Pub,Coffee Shop,Italian Restaurant,Salad Place,Thai Restaurant,Sushi Restaurant,Sandwich Place,Chinese Restaurant,Restaurant,Grocery Store
8,Barbican,Pub,Hotel,Coffee Shop,Museum,Gym / Fitness Center,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
10,Battersea Park,Pub,Italian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,Bar,Train Station,Spanish Restaurant,Ice Cream Shop,Electronics Store
11,Bayswater,Chinese Restaurant,Pub,Greek Restaurant,Hostel,Café,Indian Restaurant,Gym / Fitness Center,Coffee Shop,Hotel,Grocery Store
13,Bermondsey,Fried Chicken Joint,Pub,Coffee Shop,Ice Cream Shop,Asian Restaurant,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
19,Borough,Pub,Sandwich Place,Coffee Shop,Shoe Store,Lebanese Restaurant,Breakfast Spot,Italian Restaurant,Café,Indian Restaurant,Music Venue
21,Bow Road,Burger Joint,Pub,Bus Stop,Coffee Shop,Metro Station,Flower Shop,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
24,Brondesbury,Bus Stop,Grocery Store,Italian Restaurant,Train Station,Pizza Place,Deli / Bodega,Portuguese Restaurant,Pub,Doner Restaurant,Coffee Shop
